<a href="https://colab.research.google.com/github/moonlovist/ML_DLA/blob/main/CNN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
import os
from sklearn import preprocessing

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import  BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPool2D
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

from sklearn.metrics import roc_curve, precision_recall_curve, accuracy_score,auc

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
pathinData="/content/drive/My Drive/ML_DLA"
pathout='/content/drive/My Drive/ML_DLA/model'
model_name = '/cnn_id_SN_3_NHI_20'  #name of the final model which is saved in pathout


In [15]:
import keras
print("Predicting...")
print("====================")
LOAD_MODEL=True
if LOAD_MODEL:
  cnn = keras.models.load_model(pathout+model_name+"_best.hd5")

Predicting...


In [ ]:
X = fits.open(pathinData+'/files/Classification_catalog_SN_all_NHI_20_mock_4.7.01.fits')[0].data
X = np.expand_dims(X,axis=2)
Y1_id = fits.open(pathinData+'/files/Classification_id_SN_all_NHI_20_mock_4.7.01.fits')[1].data
Y1 = fits.open(pathinData+'/files/Classification_id_SN_all_NHI_20_mock_4.7.01.fits')[0].data

In [ ]:
Y1_id = Y1_id[Y1==1]
X = X[Y1==1]
mu = np.median(X,axis=(1,2))
X_test = X
for i in range(0,mu.shape[0]):
  X_test[i,:,:] = X[i,:,:]/mu[i]
Y_pred_all = cnn.predict(X_test)
Y_pred_class_all=Y_pred_all*0
Y_pred_class_all[Y_pred_all>0.5]=1

In [ ]:
import pandas as pd
data = pd.DataFrame({'id':Y1_id,'prediction':Y_pred_class_all.reshape(-1)})
data_mean = data.groupby('id').agg('mean')

In [ ]:
data_mean.shape[0]

16605

In [ ]:
data_mean

,prediction
id,
"(1160000499,)",0.996364
"(1164000610,)",1.000000
"(1164000621,)",0.000000
"(1166000592,)",1.000000
"(1168000608,)",0.745520
...,...
"(7296000517,)",0.965517
"(7299000447,)",0.996416
"(7300000584,)",0.000000


In [ ]:
print('completeness:',data_mean.loc[data_mean['prediction']>0.5].shape[0]/data_mean.shape[0])

completeness: 0.9394158386028305


Accuracy

In [16]:
X = fits.open(pathinData+'/files/Classification_catalog_SN_all_NHI_20_mock_4.7.01.fits')[0].data
X = np.expand_dims(X,axis=2)
Y1_id = fits.open(pathinData+'/files/Classification_id_SN_all_NHI_20_mock_4.7.01.fits')[1].data
Y1 = fits.open(pathinData+'/files/Classification_id_SN_all_NHI_20_mock_4.7.01.fits')[0].data

In [17]:
#X=X[0:len(X)//3,:,:]
#Y1_id=Y1_id[0:len(Y1_id)//3]
#Y1=Y1[0:len(Y1)//3]
X=X[len(X)//5*3:len(X)//5*4,:,:]
Y1_id=Y1_id[len(Y1_id)//5*3:len(Y1_id)//5*4]
Y1=Y1[len(Y1)//5*3:len(Y1)//5*4]

In [18]:
X.shape

(4505765, 400, 1)

In [19]:
mu = np.median(X,axis=(1,2))
X_test = X
for i in range(0,mu.shape[0]):
  X_test[i,:,:] = X[i,:,:]/mu[i]
Y_pred_all = cnn.predict(X_test)
#Y_pred = Y_pred_all.reshape(-1)*0
#Y_pred[Y_pred_all.reshape(-1)>0.5]=1

In [20]:
import pandas as pd
Y_pred = Y_pred_all.reshape(-1)*0
Y_pred[Y_pred_all.reshape(-1)>0.5]=1
Y1_new=Y1.byteswap().newbyteorder()
data = pd.DataFrame({'id':Y1_id,'prediction':Y_pred,'true':Y1_new})
data_mean = data.groupby('id').agg('mean')
print(data_mean.shape[0])
print(data_mean)
#data_mean['prediction'][data_mean['prediction']>0.5] = 1
#data_mean['prediction'][data_mean['prediction']<0.5] = 0
#data_mean['true'][data_mean['true']>0.5] = 1
#data_mean['true'][data_mean['true']<0.5] = 0


4191
               prediction      true
id                                 
(1172000610,)    0.225352  0.206820
(1174000601,)    0.000000  0.000000
(1174000614,)    0.223471  0.225968
(1176000646,)    0.026686  0.022239
(1176000708,)    0.035250  0.000000
...                   ...       ...
(7293000512,)    0.346924  0.172721
(7293000561,)    0.003049  0.000000
(7295000441,)    0.014286  0.000000
(7295000482,)    0.318755  0.206820
(7295000584,)    0.350962  0.335337

[4191 rows x 2 columns]


In [22]:
data_count = data.groupby('id').agg('count')
data_count

,prediction,true
id,,
"(1167000597,)",762,762
"(1169000563,)",1349,1349
"(1169000627,)",1161,1161
"(1169000666,)",867,867
"(1169000679,)",1121,1121
...,...,...
"(7294000596,)",1072,1072
"(7296000446,)",722,722
"(7296000464,)",1096,1096


In [21]:
print('DLA completeness:',np.count_nonzero(Y_pred[Y1==1]==Y1[Y1==1])/Y1[Y1==1].shape[0])
print('DLA purity:',np.count_nonzero(Y_pred==Y1)/Y1.shape[0])

DLA completeness: 0.8862050387067336
DLA purity: 0.9312589538069562


In [22]:
#data_mean_completeness = data_mean.loc[data_mean['true']==1]
#print('completeness:',np.count_nonzero(data_mean_completeness['prediction']==data_mean_completeness['true'])/data_mean_completeness.shape[0])
#print('purity:',np.count_nonzero(data_mean['prediction']==data_mean['true'])/data_mean.shape[0])
print('completeness:',np.count_nonzero(data_mean['prediction']>0)/data_mean.shape[0])

completeness: 0.9045573848723455


number,completeness DLA,purity, completeness catalog

In [ ]:
6696,0.8864,0.9355，0.887

In [ ]:
4198  0.8895   0.9313   0.88970